In [1]:
import sys

from sklearn.ensemble import RandomForestClassifier

sys.path.insert(0, '/home/gemeinl/code/braindecode-features/')
from braindecode_features import *

In [2]:
subject_id = 1
train_or_test = 'train'
#path = f'/home/gemeinl/code/mne-features/examples/{subject_id}/{train_or_test}/'
#path = '/home/kiessnek/braindecode_features_master/TUH/train_rec/' 
path = f'./tmp/1/{train_or_test}'
n_jobs = 2
seed = 20210408
np.random.seed(seed)
only_last_fold = True
# set to integer to enable autosklearn usage with specified integer as training time in minutes
n_min = None  # None/2/20 
drop_a_window = 0

clfs = []
rf = RandomForestClassifier(
    n_estimators=750, 
    random_state=seed
)
clfs.append(rf)
if n_min is not None:
    # optional import which raises warnings
    from autosklearn.experimental.askl2 import AutoSklearn2Classifier
    from autosklearn.classification import AutoSklearnClassifier
    asc = AutoSklearnClassifier( 
        time_left_for_this_task=60*n_min,
        ml_memory_limit=8192,
        n_jobs=n_jobs,
        seed=seed,
        initial_configurations_via_metalearning=0,
    )
    clfs.append(asc)

# read features from disk
df = read_features(
    path=path,
    n_jobs=n_jobs,
)
df.Target = df.Target.astype('int64')

# potentially drop first window (for HGD, better decoding when starting at 500ms?)
if drop_a_window is not None:
    df = drop_window(df, drop_a_window)

# try different feature usages
# TODO: fix None, False
agg_funcs = [None, np.mean]
windows_as_examples_ = [True, False]
for clf in clfs:
    # TODO: add filter_df of feature dataframe
    for agg_func, windows_as_examples in zip(agg_funcs, windows_as_examples_):
        cross_validate(
            df=df,
            clf=clf,
            subject_id=subject_id,
            only_last_fold=only_last_fold,
            agg_func=agg_func,
            windows_as_examples=windows_as_examples,
            out_path=None,
        )

                estimator agg_func  windows_as_examples  window_accuracy  \
0  RandomForestClassifier     None                 True              0.0   

   trial_accuracy  
0             0.0  
                estimator agg_func  windows_as_examples  window_accuracy  \
0  RandomForestClassifier     mean                False              1.0   

   trial_accuracy  
0             1.0  


In [3]:
results = pd.read_csv(
    '/home/gemeinl/code/mne-features/examples/1/cv_results.csv',
    index_col=0
).reset_index(drop=True)

In [4]:
results.columns

Index(['subject', 'fold', 'estimator', 'window_accuracy', 'trial_accuracy',
       'predictions', 'targets', 'model', 'feature_names',
       'windows_as_examples', 'agg_func'],
      dtype='object')

In [6]:
results[['subject', 'estimator', 'window_accuracy', 'trial_accuracy', 'agg_func', 'windows_as_examples']]

,subject,estimator,window_accuracy,trial_accuracy,agg_func,windows_as_examples
0,1,RandomForestClassifier,0.753125,0.859375,NaN,True
1,1,RandomForestClassifier,0.921875,0.921875,NaN,False
2,1,RandomForestClassifier,0.906250,0.906250,mean,False
